<h1> Generador de archivos publicaciones por autor Google Scholar </h1>

Este notebook contiene un script para generar listados de publicaciones para un autor de acuerdo a lo indexado en Google Scholar. El archivo generado tiene las siguientes características
1. Está en formato csv con el nombre del autor buscado
2. Se guarda automáticamente en la carpeta en que está el notebook
3. Contiene la producción buscada por autor ordenada por fecha
4. No incluye hipervínculos de descarga

In [1]:
#carga de librerías

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
import pandas as pd

In [2]:
#opciones de navegación
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--disable-extensions')
driver_path = 'C:\\Users\\Carlos\\Downloads\\chromedriver_win32\\chromedriver.exe'

En la siguiente línea se debe indicar el nombre del autor a buscar, en el formato Nombre+Apellido

In [3]:
buscar = "Vicente+Sisto"

In [4]:
#iniciamos el driver

driver = webdriver.Chrome(driver_path, chrome_options = options)

#aquí declaramos la web a la que entraremos

driver.get('https://scholar.google.com/citations?view_op=search_authors&mauthors={}&hl=es&oi=drw'.format(buscar))

#iniciamos el proceso de búsqueda

WebDriverWait(driver, 5)\
    .until(EC.element_to_be_clickable((By.CSS_SELECTOR,
                                     'span.gs_hlt')))\
    .click()

WebDriverWait(driver, 5)\
    .until(EC.element_to_be_clickable((By.CSS_SELECTOR,
                                     'span#gsc_a_ha.gsc_a_h')))\
    .click()

#esto es para mostrar 20 registros más
WebDriverWait(driver, 8)\
    .until(EC.element_to_be_clickable((By.CSS_SELECTOR,
                                     'button#gsc_bpf_more.gs_btnPD.gs_in_ib.gs_btn_flat.gs_btn_lrge.gs_btn_lsu')))\
    .click()

#este sleep permite que se carguen correctamente los registros adicionales antes de capturarlos en una variable
#una alternativa es elements = WebDriverWait(driver, 20).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, ".content")))

time.sleep(3)

#en este caso, debemos hacer el split con "\n" porque es un caracter no imprimible de LineFeed
#aquí presentan cómo se usa https://www.geeksforgeeks.org/python-string-splitlines-method/
#esto permite visualizar los caracteres unicode no imprimibles https://www.soscisurvey.de/tools/view-chars.php

#declaramos como lista las variables que serán las columnas del data frame, la lista de títulos, autores, referencias y año

titulos_lista = list()
autores_lista = list()
referencia_lista = list()
año_lista = list()

#hacemos la búsqueda por XPATH, la alternativa era buscar por CLASS, pero se prefirió esta porque funciona más rápido aquí

titulos = driver.find_elements(By.XPATH, '/html/body/div/div[12]/div[2]/div/div[4]/form/div[1]/table/tbody/tr/td/a')
for i in range (0, len(titulos), 2):
    titulos_lista.append(titulos[i].text)

autores = driver.find_elements(By.XPATH, '/html/body/div/div[12]/div[2]/div/div[4]/form/div[1]/table/tbody/tr/td/div[1]')
for a in range (0, int(len(autores))):    
    autores_lista.append(autores[a].text)
    
referencia = driver.find_elements(By.XPATH, '/html/body/div/div[12]/div[2]/div/div[4]/form/div[1]/table/tbody/tr/td/div[2]')
for r in range (0, int(len(referencia))):    
    referencia_lista.append(referencia[r].text)

año = driver.find_elements(By.XPATH, '/html/body/div/div[12]/div[2]/div/div[4]/form/div[1]/table/tbody/tr/td[3]')
for a in range (0, len(año)):
    año_lista.append(año[a].text)

#generamos un dataframe con las listas anteriores en forma de columnas

revision = pd.DataFrame({"título" : titulos_lista, "autores" : autores_lista, "referencia" : referencia_lista, "año" : año_lista})

#exportamos a csv usando la clave buscada como nombre de archivo

revision.to_csv('revision_{}.csv'.format(buscar), encoding='utf-8')

#cerramos el driver y la ventana de Chrome

driver.close()
driver.quit()


C:\Users\Public\Documents\Wondershare\CreatorTemp/ipykernel_15300/2133894873.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path, chrome_options = options)
C:\Users\Public\Documents\Wondershare\CreatorTemp/ipykernel_15300/2133894873.py:3: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(driver_path, chrome_options = options)


In [5]:
#con esta línea podemos visualizar aquí el dataframe

revision

,título,autores,referencia,año
0,Experiencias y resistencias de los y las profe...,"LRC del Valle, E Baleriola, V Sisto","Revista Izquierdas, 14",2022
1,The ethics of public work and the impertinence...,"V Sisto, L Ramírez-Casas del Valle, L Núñez-Pa...",,2021
2,La ética de lo público y la impertinencia del ...,"V Sisto, L Ramírez-Casas del Valle, L Núñez-Pa...","Psicoperspectivas 20 (3), 151-162",2021
3,Entre managerialismo y nostalgia. El debate po...,"V Sisto, L Núñez-Parra, G Rivera-Aguilera","Quaderns de Psicologia 23 (2), e1695-e1695",2021
4,Estudiar las transformaciones contemporáneas d...,"V Sisto, C Fardella",Quaderns of Psicologia 23 (2),2021
...,...,...,...,...
95,Control narrativo y gubernamentalidad: la prod...,"V Sisto, C Fardella",Forum Qualitative Sozialforschung / Forum: Qua...,2009
96,Identidades en Disputa: las Identidades Labora...,"V SISTO, C FARDELLA, L AHUMADA, C MONTECINOS",Pontificia Universidad Católica de Valparaíso....,2009
97,Nuevo Management Público en Chile e Identidad,"V Sisto, C Fardella, L Ahumada, C Montecinos",,2009
98,Control narrativo y gubernamentalidad: La prod...,"V Sisto, C Fardella",El caso de profepresentada en Congreso Pre Ala...,2009
